### Prototype for an RFX Answering AI Assistant

* read question from an **xls** file
* answers based on a **KNOWLEDGE BASE**: a set of documents saved in a Vector Store
* manage the **history** of questions
* provides the list of references
* it is possible to change prompt, to improve answers
* if a reranker is used provides relevance scores for refs
* can use LLMLingua as prompt compressor

Additional info:
* Using the Vector Store configured in config.py
* if you want to improve changing prompts, change in **oracle_chat_prompts.py**. Maybe you need to change only the second prompt!

In [1]:
from pathlib import Path

# to read, write xls
import pandas as pd

from langchain_core.messages import HumanMessage, AIMessage

# my code to simplify use of LangChain
# this function will return a contextualized RAG chain
# using chat_history

# changed to use the forked for rfx
from factory_rfx import build_rag_chain

from utils import print_configuration, get_console_logger, remove_path_from_ref

# config (some config are read by get_rag_chain())
from config_rfx import DO_STREAMING, ADD_RERANKER, ADD_REFERENCES

Notes:
* If you don't have a LangSmith account, set in config.py ENABLE_TRACING = **False**

In [2]:
# Settings for input/output

# the input file is here...
INPUT_DIR = Path(".")
# output file in this directory
OUTPUT_DIR = Path(".")

# Input: we take the questions from this file
QUERY_FILE = "rfp01.xlsx"

# the name of the column with all questions
QUESTION_COL_NAME = "Question"

# full path of questions file
QUERY_PATH_NAME = INPUT_DIR / QUERY_FILE

# we write results to this file
OUTPUT_FILE_NAME = "answers01.xlsx"
OUTPUT_PATH_NAME = OUTPUT_DIR / OUTPUT_FILE_NAME

logger = get_console_logger()

# Num. of questions to answer (only to limit time in regression test)
# put a very high value to answer ALL
NUM_Q_TO_ANSWER = 2

In [3]:
# check that streaming has been disabled
# this NB must be run with False
assert DO_STREAMING is False

In [4]:
# some helper functions
def format_metadata(metadata):
    """
    input: metadata is a dict
    """
    # remove path from source
    metadata["source"] = remove_path_from_ref(metadata["source"])

    # we can add the relevance score only if there is a reranker
    MAX_DEC_DIGITS = 4

    # reduce num of digits in the relevance score
    if ADD_RERANKER:
        metadata["relevance_score"] = round(metadata["relevance_score"], MAX_DEC_DIGITS)

        result_string = f"Source: {metadata['source']} pag.: {metadata['page']}, score: {metadata['relevance_score']}"
    else:
        # no reranker, remove the score
        # with LLMLingua no relevance score
        result_string = f"Source: {metadata['source']} pag.: {metadata['page']}"

    return result_string


def print_references(response_ai_msg):
    """
    format refs lines and remove duplicates (due to chunking)

    added error handling (source not found?)
    """
    list_bookname_page = []

    print("")
    print("References:")

    try:
        for doc in response_ai_msg["context"]:
            book_name_page = doc.metadata["source"] + str(doc.metadata["page"])

            if book_name_page not in list_bookname_page:
                print(format_metadata(doc.metadata))

                # register it
                list_bookname_page.append(book_name_page)

        print("")
    except Exception as e:
        # go over
        print("......")

#### Read the input

In [5]:
input_df = pd.read_excel(QUERY_PATH_NAME)

questions = list(input_df[QUESTION_COL_NAME].values)

# a look at the first five questions

logger.info(f"There are {len(questions)} questions...")
logger.info("")
logger.info("First 5 questions:")
logger.info(questions[:5])

2024-05-26 19:26:31,733 - There are 12 questions...
2024-05-26 19:26:31,734 - 
2024-05-26 19:26:31,734 - First 5 questions:
2024-05-26 19:26:31,734 - ['What is the last release for Oracle Database?', 'Is it a long term release support?', 'What functionalities Oracle DB provides for High Availability?', 'What functionalities Oracle DB provides for JSON Support?', 'List new functionalities regarding JSON']


#### Init the RAG chain and Chat History

In [6]:
# here we will put input and outputs to give context
# to reset chat, simply empty the list
chat_history = []

# here we will save answers, to create the output file
answers = []

# we create the RAG chain with LangChain
rag_chain = build_rag_chain(None, None, verbose=True)

2024-05-26 19:26:33,492 - --------------------------------------------------
2024-05-26 19:26:33,493 - Configuration used:
2024-05-26 19:26:33,494 - 
2024-05-26 19:26:33,495 -  Embedding model type: OCI
2024-05-26 19:26:33,496 -  Using cohere.embed-multilingual-v3.0 for Embeddings...
2024-05-26 19:26:33,497 -  Added Cohere Reranker...
2024-05-26 19:26:33,498 -  Using rerank-multilingual-v3.0 as reranker...
2024-05-26 19:26:33,499 -  Using OPENSEARCH as Vector Store...
2024-05-26 19:26:33,500 -  Retrieval parameters:
2024-05-26 19:26:33,500 -     TOP_K: 8
2024-05-26 19:26:33,502 -     TOP_N: 4
2024-05-26 19:26:33,503 -  Using COHERE as Generative Model type...
2024-05-26 19:26:33,504 -  Using command-r for LLM...
2024-05-26 19:26:33,504 - 
2024-05-26 19:26:33,505 -  Enabled Observability with LangSmith...
2024-05-26 19:26:33,505 - --------------------------------------------------
2024-05-26 19:26:33,506 - 
2024-05-26 19:26:33,655 - Adding a reranker...


#### Answer all the questions

In [7]:
%%time

# loop all over the questions, call RAG chain, save answer
for i, question in enumerate(questions):
    if i > NUM_Q_TO_ANSWER:
        # exit
        break

    # print the header of the section
    print("------------------------------------------------------------------")
    print("")
    print(f"Question n. {i+1}: {question}:")
    print()

    try:

        # to give context for the answers we need to save questions
        # and answers in the chat history as Messages, with the right roles
        chat_history.append(HumanMessage(content=question))

        # the invocation to the RAG chain, with the history

        # can add instructions this way
        # question += " Rispondi in italiano."

        input_msg = {
            "input": question,
            "chat_history": chat_history,
        }

        # we need to check if we have setup streaming mode,
        # for the right invocation
        if DO_STREAMING:
            ai_msg = rag_chain.stream(input_msg)
        else:
            ai_msg = rag_chain.invoke(input_msg)

        # extract only the answer
        answer = ai_msg["answer"]

        # print the answer
        print(answer)

        # print the link to refs (chunks of text used as context for answers)
        if ADD_REFERENCES:
            print_references(ai_msg)

        # save the answer
        answers.append(answer)

        # save in the msg history
        chat_history.append(AIMessage(content=answer))
    except Exception as e:
        # got an error, try to continue
        print("")
        print("Got unexpected error.. try to continue...")
        print("")

        answers.append("Not available...")

------------------------------------------------------------------

Question n. 1: What is the last release for Oracle Database?:

According to the provided information, the latest release of Oracle Database is Oracle Database 23ai, released on May 13, 2024. It is documented in the Oracle Database New Features Release 23ai.

References:
Source: oracle-database-23ai-new-features-guide.pdf pag.: 0, score: 0.8818
Source: oracle-database-23c-new-features-guide.pdf pag.: 0, score: 0.861
Source: oracle-database-23ai-new-features-guide.pdf pag.: 3, score: 0.7576
Source: database-concepts.pdf pag.: 337, score: 0.0003

------------------------------------------------------------------

Question n. 2: Is it a long term release support?:

Yes, according to the information I have access to, Oracle Database 23ai and Oracle Database 23c are both described as long-term support releases. They are the current most up-to-date versions of Oracle Database, building upon the extensive features of Oracle Da

#### Save the output file

In [ ]:
# using Pandas, maybe will find a better way
dict_out = {"Question": questions, "Answer": answers}

output_df = pd.DataFrame(dict_out)

logger.info(f"Writing answers to: {OUTPUT_PATH_NAME}")

output_df.to_excel(OUTPUT_PATH_NAME, index=None)